In [ ]:
import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#keras에서 제공하는 이미지 제너레이터 사용 - 데이터 증식을 위해 사용
#이미지의 위치가 조금 바뀌거나 회전 좌우반전등을 했을 때 컴퓨터가 받아들이는 이미지 가 전혀 다른것이 되기 때문에 
#이러한 변형을 줌으로써 학습데이터를 늘리고 이러한 변조에 강하게 모델을 학습시킬수 있다. 
# 위노그라드 알고리즘 설정
#os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

rootPath = 'C:/Users/PMJ/Desktop/dataset/cat-and-dog'

imageGenerator = ImageDataGenerator(
    rescale=1./255, #이미지의 nomalization을 위해 사용 이미지별로 255로 나눈 값으로 데이터가 변형 
    rotation_range=20, #이미지의 최대 회전각을 지정 최대 20도까지 회전 
    width_shift_range=0.1, #이미지의 이동을 말함 좌우/상하 이미지의 이동하는 백분율을 지정합니다. 
    height_shift_range=0.1, 
    brightness_range=[.2, .2], #이미지 밝기 내용
    horizontal_flip=True, #이미지 수평 반전 시켜준다. 반례  : MNIST 경우 수평반전을 사용하면 안된다. 
    validation_split=.1 #검증세트의 비율을 지정
)

trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(64, 64),
    subset='training'
)

validationGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(64, 64),
    subset='validation'
)


#모델 구성 

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()

model.add(layers.InputLayer(input_shape=(64, 64, 3)))
model.add(layers.Conv2D(16, (3, 3), (1, 1), 'same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.3))

model.add(layers.Conv2D(32, (3, 3), (1, 1), 'same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.3))

model.add(layers.Conv2D(64, (3, 3), (1, 1), 'same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.3))


model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.summary()

trainGen.samples

model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['acc'], #metrics 확률 
)

epochs = 32
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / epochs,
)

import matplotlib.pyplot as plt

def show_graph(history_dict):
    accuracy = history_dict['acc']
    val_accuracy = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(16, 1))
    
    plt.subplot(121)
    plt.subplots_adjust(top=2)
    plt.plot(epochs, accuracy, 'ro', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
    plt.title('Trainging and validation accuracy and loss')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy and Loss')

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5)
#     plt.legend(bbox_to_anchor=(1, -0.1))

    plt.subplot(122)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=5)
#     plt.legend(bbox_to_anchor=(1, 0))

    plt.show()


testGenerator = ImageDataGenerator(
    rescale=1./255
)

testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(64, 64),
)

model.evaluate_generator(testGen)
epochs = 1000
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / epochs,
)

show_graph(history.history)


from tensorflow.keras.preprocessing.image import array_to_img
import numpy as np

cls_index = ['새', '개']

imgs = testGen.next()
arr = imgs[0][0]
img = array_to_img(arr).resize((128, 128))
plt.imshow(img)
result = model.predict_classes(arr.reshape(1, 64, 64, 3))
print('예측: {}'.format(cls_index[result[0]]))
print('정답: {}'.format(cls_index[np.argmax(imgs[1][0])]))
model.evaluate_generator(testGen)
